In [1]:
import pandas as pd
from sells_view import plot_results

In [2]:
df = pd.read_csv("BTC_ETH_LTC_Mar0522_Mar1022.csv")

In [3]:
df.head()

,OpenTime,ADA-USD_Open,ADA-USD_High,ADA-USD_Low,ADA-USD_Close,ADA-USD_volume,CloseTime,ADA-QuoteAssetVolume,ADA-NumberOfTrades,ADA-TBBAV,...,BTC-USD_Open,BTC-USD_High,BTC-USD_Low,BTC-USD_Close,BTC-USD_volume,BTC-QuoteAssetVolume,BTC-NumberOfTrades,BTC-TBBAV,BTC-TBQAV,BTC-ignore
0,2022-01-25 14:00:00,1.035,1.042,1.022,1.030,8240060.4,2022-01-25 14:59:59.999,8.504572e+06,15557.0,4266813.6,...,36430.38,36600.44,36227.52,36406.00,1671.86116,6.086462e+07,49973.0,829.37520,3.019401e+07,0.0
1,2022-01-25 15:00:00,1.031,1.040,1.013,1.029,11912749.4,2022-01-25 15:59:59.999,1.223172e+07,22446.0,5954033.6,...,36406.00,36777.77,36201.68,36519.13,2239.52587,8.170238e+07,58332.0,1203.30230,4.390835e+07,0.0
2,2022-01-25 16:00:00,1.030,1.033,1.020,1.031,6338242.5,2022-01-25 16:59:59.999,6.504304e+06,11686.0,3245062.4,...,36519.12,36579.99,36259.10,36498.00,2008.17575,7.311807e+07,63309.0,1000.52554,3.642950e+07,0.0
3,2022-01-25 17:00:00,1.031,1.053,1.017,1.019,16720492.1,2022-01-25 17:59:59.999,1.732742e+07,32455.0,8297087.4,...,36497.39,37149.42,36252.00,36299.99,5493.26524,2.014882e+08,130139.0,2888.12575,1.059381e+08,0.0
4,2022-01-25 18:00:00,1.020,1.039,1.003,1.028,16961424.2,2022-01-25 18:59:59.999,1.733301e+07,28019.0,8411770.5,...,36299.99,36780.00,35967.33,36530.00,3943.20294,1.434593e+08,86662.0,1954.36935,7.109640e+07,0.0


In [4]:
def sma(data, window):
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std * nstd
    lower_band = sma - std * nstd
    return upper_band, lower_band

In [5]:
symbols = ['BTC', 'ETH', 'LTC', 'ANC', 'ADA']

In [6]:
nstd = 3

for symbol in symbols:
    df[f'{symbol}_sma'] = sma(df[f'{symbol}-USD_Open'], 20)
    df[f'{symbol}_upper_band'], df[f'{symbol}_lower_band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_sma'], 20, nstd)

In [7]:
df.dropna(inplace=True)

In [8]:
df[['LTC-USD_Open', 'ETH-USD_Open', 'BTC-USD_Open']]

,LTC-USD_Open,ETH-USD_Open,BTC-USD_Open
19,109.6,2475.97,37600.01
20,108.4,2451.31,37288.56
21,108.5,2461.17,37338.60
22,108.9,2487.64,37591.42
23,109.2,2489.40,37650.01
...,...,...,...
1521,129.5,3429.65,47701.49
1522,129.0,3397.53,47477.85
1523,129.0,3375.40,47270.19
1524,129.2,3386.28,47331.13


In [9]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, trading_fee_mulpiplier):
        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.buys = []
        self.sells = []
        self.trading_fee_multiplier = trading_fee_mulpiplier

    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount/buy_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, buy_price])

    def sell(self, sell_price, time):
        self.balance_amount = self.balance_amount * sell_price * self.trading_fee_multiplier
        self.sells.append([self.balance_unit, time, sell_price])
        self.balance_unit = 'USDT'


In [10]:
env = TradingEnv(balance_amount=100,balance_unit='USDT', trading_fee_mulpiplier=0.999)

In [11]:
for i in range(len(df)):
    if env.balance_unit == 'USDT':
        for symbol in symbols:
            if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i]:
                env.buy(symbol, df[f'{symbol}_lower_band'].iloc[i], df['OpenTime'].iloc[i])
                break
    if env.balance_unit != 'USDT':
        if df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i]:
                env.sell(df[f'{env.balance_unit}_upper_band'].iloc[i],df['OpenTime'].iloc[i])
if env.balance_unit != 'USDT':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['OpenTime'].iloc[-1])

In [12]:
print(f'num buys: {len(env.buys)}')
print(f'num sells: {len(env.sells)}')
print(f'ending balance: {env.balance_amount} {env.balance_unit}')

num buys: 22
num sells: 22
ending balance: 71.61068480179055 USDT


In [13]:
list(zip(env.buys, env.sells))

[(['BTC', '2022-01-27 00:00:00', 36343.18957629574],
  ['BTC', '2022-01-28 23:00:00', 37787.03604678621]),
 (['BTC', '2022-01-29 19:00:00', 37325.14989070897],
  ['BTC', '2022-01-29 23:00:00', 38175.3010220776]),
 (['BTC', '2022-01-30 22:00:00', 37603.5501828559],
  ['BTC', '2022-01-31 19:00:00', 38284.07723826924]),
 (['ANC', '2022-02-02 06:00:00', 1.5294150252491245],
  ['ANC', '2022-02-04 09:00:00', 1.3591499552772488]),
 (['BTC', '2022-02-06 17:00:00', 41320.39339365895],
  ['BTC', '2022-02-06 17:00:00', 41819.900606341056]),
 (['BTC', '2022-02-06 18:00:00', 41258.287950248385],
  ['BTC', '2022-02-07 02:00:00', 41863.289450185446]),
 (['BTC', '2022-02-08 15:00:00', 43156.11063984816],
  ['BTC', '2022-02-09 22:00:00', 44699.37306047699]),
 (['ETH', '2022-02-10 16:00:00', 3126.9657728317393],
  ['ETH', '2022-02-14 15:00:00', 2922.5039777748066]),
 (['BTC', '2022-02-16 07:00:00', 43694.4779374992],
  ['BTC', '2022-02-21 04:00:00', 39360.0066948592]),
 (['BTC', '2022-02-21 14:00:00', 3

In [14]:
plot_results(df, 'BTC', env.buys, env.sells)

KeyError: 'BTC_sma-25'